In [1]:
import argparse

import torch
import timm
import torchvision
from torch.utils.data import DataLoader

from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint

from misc import collate_fn, Augment_v2
from model import MoCo


In [2]:
model = timm.create_model('vit_base_patch16_224', pretrained=False, img_size=224)

In [3]:

import torch.nn.functional as F

q1 = torch.tensor([(1,2,3),(1,2,3),(1,2,3)], dtype=torch.float32)
z1 = torch.tensor([(-4,1,2),(0,5,3),(1,2,3)], dtype=torch.float32)
q1.shape
# z1 = torch.tensor((2,2,3))

# q1 = torch.randn(10,10)
# z1 = torch.randn(10,10)
p = F.normalize(q1, dim=1)
z = F.normalize(z1, dim=1)

2 - 2 * (p * z).sum(dim=1).mean()

tensor(0.5972)

In [4]:
(p * z).sum(dim=1)

tensor([0.2333, 0.8709, 1.0000])

In [5]:
torch.matmul(p, z.T)

tensor([[0.2333, 0.8709, 1.0000],
        [0.2333, 0.8709, 1.0000],
        [0.2333, 0.8709, 1.0000]])

In [6]:

p = F.normalize(p, dim=1)
z = F.normalize(z, dim=1)
2 - 2 * (p * z).sum(dim=1).mean()

tensor(0.5972)

In [15]:
x = torch.arange(100)
x = x.reshape(10,10)
n, m = x.shape
x

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
        [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
        [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [20]:
x.flatten()[:-1].view(n - 1, n + 1)[:,1:].flatten()

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39,
        40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59,
        60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79,
        80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98])

In [ ]:
# x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()